# Cell Nuclei

In this example, we will use a cellular image from the Allen Cell WTC-11 hiPSC Single-Cell Image Dataset ([Viana et al. 2023](https://doi.org/10.1038/s41586-022-05563-7)).

In [1]:
import libcarna
import skimage

Get the data:

In [2]:
data = skimage.data.cells3d()
data.shape

(60, 2, 256, 256)

The data has 60 slices and 2 channels. We will visualize the nuclei channel of the data.

## Maximum Intensity Projection

In the code below, we use `normals=True` on the `volume` node; albeit this doesn't make any difference for the *Maximum
Intensity Projection* (MIP), it is benefitial for other, subsequently shown rendering modes.

In [3]:
GEOMETRY_TYPE_VOLUME = 2

# Create and configure frame renderer
mip = libcarna.mip(GEOMETRY_TYPE_VOLUME, cmap='jet', sr=500)
r = libcarna.renderer(600, 450, [mip])

# Create and configure scene
root = libcarna.node()
libcarna.volume(
    GEOMETRY_TYPE_VOLUME,
    data[:, 1],
    parent=root,
    spacing=(1, 0.5, 0.5),
    local_transform=libcarna.rotate('y', 90).rotate('x', -35),
    normals=True,
)
camera = libcarna.camera(
    parent=root,
    projection=r.frustum(fov=90, z_near=1, z_far=500),
    local_transform=libcarna.translate(0, 0, 100),
)

# Render
libcarna.imshow(r.render(camera))

In the MIP, it can easily be seen that there is one mitotic nucleus in the image.

For an even better visual perception of the 3D data, it is best viewed from different angles, that can be achieved with
as a subtle animation:

In [4]:
# Render as animation
libcarna.imshow(
    libcarna.animate(
        libcarna.animate.swing_local(camera, amplitude=22),
        n_frames=50,
    ).render(r, camera),
)

## Direct Volume Rendering

In a *Direct Volume Rendering* (DVR), surfaces are rendered by simulation of the absorption of light. This simulation
is most realstic, when the spatial orientation of the surfaces can be taken into account, which requires that the
normals of the volume have been computed (this is why we used `normals=True` when we created the `volume` node).

In [5]:
dvr = libcarna.dvr(
    GEOMETRY_TYPE_VOLUME, sr=500, transl=0, diffuse=0.8,
)

We use a custom colormap, that employs blueish colors for the nuclear envelope, and reddish colors for the chromatin:

In [6]:
dvr.cmap.clear()
dvr.cmap.write_linear_segment( # blueish colors
    0.1, 0.3,
    libcarna.color(0,   0, 255,   0),
    libcarna.color(0, 255, 255, 255),
)
dvr.cmap.write_linear_segment( # reddish colors
    0.3, 0.9,
    libcarna.color(0,   255, 255, 255),
    libcarna.color(255, 255,   0, 255),
)

libcarna.imshow(
    libcarna.animate(
        libcarna.animate.swing_local(camera, amplitude=22),
        n_frames=50,
    ).render(
        libcarna.renderer(600, 450, [dvr]),
        camera,
    ),
)

The DVR of the nuclei in the image allows for a very natural perception of the 3D scene. On the downside, the mitotic nucleus is harder to identify. This is because the chromatin from the inside of the nucleus (should be reddish due to our colormap) is still fully occluded by the nuclear envelope (blueish).

We can overlay a DVR of the nuclear envelope with a MIP for the chromatin:

In [7]:
mip = libcarna.mip(GEOMETRY_TYPE_VOLUME, sr=500)
mip.cmap.clear()
mip.cmap.write_linear_segment(
    0.3, 0.7,
    libcarna.color(0, 255, 255,   0),
    libcarna.color(255, 255, 0, 255),
)
mip.cmap.write_linear_segment(
    0.7, 1.0,
    libcarna.color(255, 255, 0, 255),
    libcarna.color(255,   0, 0, 255),
)

dvr = libcarna.dvr(
    GEOMETRY_TYPE_VOLUME, sr=500, transl=0, diffuse=0.8,
)
dvr.cmap.clear()
dvr.cmap.write_linear_segment(
    0.1, 0.3,
    libcarna.color(0,   0, 255,   0),
    libcarna.color(0, 255, 255, 255),
)

libcarna.imshow(
    libcarna.animate(
        libcarna.animate.swing_local(camera, amplitude=22),
        n_frames=50,
    ).render(
        libcarna.renderer(600, 450, [dvr, mip]),
        camera,
    ),
)